# Import packages, read files and create empty dicts

In [1]:
import os, re, sys, flopy
import numpy as np
np.set_printoptions(precision=4, suppress=True)

sys.path.insert(0, './VTKGeneration')

from vtkProcess import vtkProcess
from vtkFunctions import vtkFunctions  

In [2]:
modPath = '../Model/'
modName = 'Model1'
exeName = '../Exe/MODFLOW-NWT_64.exe'  
modelObject = flopy.modflow.Modflow.load(modName+'.nam', model_ws=modPath, 
                                exe_name=exeName)

In [3]:
modelObject.get_package_list()

['DIS', 'NWT', 'BAS6', 'UPW', 'RCH', 'EVT', 'DRN', 'OC']

In [4]:
disObject = modelObject.get_package('DIS') #Model Geometry
basObject = modelObject.get_package('BAS6') #Starting heads and active zone
#lpfObject = modelObject.get_package('LPF') #K distribution
upwObject = modelObject.get_package('UPW') #K distribution
#chdObject = modelObject.get_package('CHD')
drnObject = modelObject.get_package('DRN')
#welObject = modelObject.get_package('WEL')
#rivObject = modelObject.get_package('RIV')
#rchObject
#evtObject
#hdsObject = flopy.utils.formattedfile.FormattedHeadFile(modPath+modName+'.hds').get_data()
hdsObject = flopy.utils.binaryfile.HeadFile(modPath+modName+'.hds').get_data()
#ucnObject = flopy.utils.binaryfile.UcnFile(modPath+modName+'_Sulphates.ucn').get_data(totim=28695526.0)

In [5]:
#create a empty dictionay to store the model features
modDis  = {}
modBas  = {}
modLpf  = {}

modDrn  = {}
modWel  = {}
#modRiv  = {}

modHds  = {}
#modUcn  = {}

<br/>
<br/>
<br/>
___

# Working with the DIS (Discretization Data) data

### General model features as modDis dict

In [6]:
modDis["cellRows"] = disObject.nrow
modDis["cellCols"] = disObject.ncol
modDis["cellLays"] = disObject.nlay
modDis["vertexLays"] = disObject.nlay + 1
modDis["vertexRows"] = modDis["cellRows"] + 1
modDis["vertexCols"] = modDis["cellCols"] + 1
modDis["vertexPerLay"] = modDis["vertexRows"] * modDis["vertexCols"]
modDis["cellsPerLay"] = modDis["cellRows"] * modDis["cellCols"]

In [7]:
modDis['DELRArray1D'] = list(disObject.delr.array)
modDis['DELCArray1D'] = list(disObject.delc.array)

In [8]:
modDis['cellZVertexGrid']={}
modDis['cellZVertexGrid']['lay0']=list(disObject.top.array.flatten())
for lay in range(modDis["cellLays"]):
    modDis['cellZVertexGrid']['lay'+str(lay+1)]=list(disObject.botm.array.flatten()
                                                     [lay*modDis["cellsPerLay"]:(lay+1)*modDis["cellsPerLay"]])

In [9]:
########################
### Geolocation model data
modDis["vertexXmin"]=0                            #disObject.xorigin.get_data()
modDis["vertexYmin"]=0                            #disObject.yorigin.get_data()
modDis["vertexXmax"]=sum(modDis['DELRArray1D'])
modDis["vertexYmax"]=sum(modDis['DELCArray1D'])

In [10]:
########################
### List of arrays of cells and vertex coord
modDis['vertexEastingArray1D']  = np.array([modDis['vertexXmin']+np.sum(modDis['DELRArray1D'][:col]) 
                                            for col in range(modDis['vertexCols'])])
modDis['vertexNorthingArray1D'] = np.array([modDis['vertexYmax']-np.sum(modDis['DELCArray1D'][:row]) 
                                            for row in range(modDis['vertexRows'])])

modDis['cellEastingArray1D']    = np.array([modDis['vertexXmin']+np.sum(modDis['DELRArray1D'][:col])+
                                            modDis['DELRArray1D'][col]/2 for col in range(modDis['cellCols'])])
modDis['cellNorthingArray1D']   = np.array([modDis['vertexYmax']-np.sum(modDis['DELCArray1D'][:row])-
                                            modDis['DELCArray1D'][row]/2 for row in range(modDis['cellRows'])])

########################
### Grid of XYZ Vertex Coordinates
modDis['vertexXGrid'] = np.repeat(modDis['vertexEastingArray1D'].reshape(modDis['vertexCols'],1),
                                  modDis['vertexRows'],axis=1).T
modDis['vertexYGrid'] = np.repeat(modDis['vertexNorthingArray1D'],
                                  modDis['vertexCols']).reshape(modDis['vertexRows'],modDis['vertexCols'])
modDis['vertexZGrid'] = vtkProcess.interpolateCelltoVertex(modDis,'cellZVertexGrid')

<br/>
<br/>
<br/>
___

# Get the Info for BAS - Active Zone

In [11]:
modBas['active'] = list(basObject.ibound.array)

<br/>
<br/>
<br/>
___

# Get the Info for LPF - Boundary Conditions and Cell Heads

In [12]:
# Get the LFP Info
modLpf['hkList'] = []
modLpf['vkaList'] = []

hkList=list(upwObject.hk.array.flatten())
vkaList=list(upwObject.vka.array.flatten())
activeList=list(np.concatenate(modBas['active']).flatten())

for index, cell in enumerate(activeList):
    if cell == 1:
        modLpf['hkList'].append(hkList[index])
        modLpf['vkaList'].append(vkaList[index])

<br/>
<br/>
<br/>
___

# Get the Info for GHB, WEL y RIV

In [13]:
# Get the GHD Info
modDrn['drnCells'] = drnObject.stress_period_data.get_dataframe()[['k','i','j','elev0']].values.tolist()

# Get the WEL Info
#modWel['welCells'] = welObject.stress_period_data.get_dataframe()[['k','i','j','flux0']].values.tolist()

# Get the RIV Info
#modRiv['rivCells'] = rivObject.stress_period_data.get_dataframe()[['k','i','j','stage0']].values.tolist()

<br/>
<br/>
<br/>
___

# Get the Info for Model Heads and WaterTable

In [14]:
# Get head for cell and vertex   
modHds['cellHeadsList'] = list(hdsObject.flatten())
modHds['vertexHeadList'] = vtkProcess.simplifiedVertexHead(modDis,hdsObject)

In [15]:
#Get the water table as function
wtObject = vtkProcess.arrayWaterTableObject(modDis,hdsObject)

<br/>
<br/>
<br/>
___

# Get the Info for Concentrations

In [16]:
# Get head for cell and vertex   
#modUcn['cellUcnList'] = list(ucnObject.flatten())
#modUcn['vertexUcnList'] = vtkProcess.simplifiedVertexHead(modDis,ucnObject)

<br/>
<br/>
<br/>
___

# VTK file of Model Geometry, Model Results and Boundary Conditions


## Point Data

In [17]:
### Vertex Heads and Concentrations
listVertexHead = modHds['vertexHeadList']
#listVertexUcn = modUcn['vertexUcnList']

### Water Tables Vextex
listWaterTableVertex = vtkProcess.listWaterTableVertexFunction(modDis,wtObject)

## Point Definition

In [18]:
### Definition of XYZ points for All Vertex 
vertexXYZPoints = vtkProcess.vertexXYZPointsFunction(modDis)

### Definition of XYZ points for Water Table
vertexWaterTableXYZPoints = vtkProcess.vertexWaterTableXYZPointsFunction(listWaterTableVertex,modDis)

## Quad and Hexa Sequences

In [19]:
### List of Layer Quad Sequences (Works only for a single layer)
listLayerQuadSequence = vtkProcess.listLayerQuadSequenceFunction(modDis,modBas,wtObject)

### List of Hexa Sequences for All Cells
listHexaSequence = vtkProcess.listHexaSequenceFunction(modDis,modBas)

### List of Hexa Sequences for GHB Cells
listDrnCellsHexaSecuence = vtkProcess.bcCellsListFunction(modDrn,'drnCells',listHexaSequence,modDis,modBas)[1]

### List of Hexa Sequences for wEL Cells
#listWelCellsHexaSecuence = vtkProcess.bcCellsListFunction(modWel,'welCells',listHexaSequence,modDis,modBas)[1]

### List of Hexa Sequences for RIV Cells
#listRivCellsHexaSecuence = vtkProcess.bcCellsListFunction(modRiv,'rivCells',listHexaSequence,modDis,modBas)[1]

## Cell Data

In [20]:
### Definition of cellK
listCellHK = modLpf['hkList']
listCellVKA = modLpf['vkaList']

### Definition of cellHead
listCellHead = [a for a in modHds['cellHeadsList'] if a>-2e+20 ]

### Definition of cellConcentrations
#listCellUcn = [a for a in modUcn['cellUcnList'] if a>-3.e+30 ]

### Definition of CHD cells values '1' as List
listDrnCellsIO = vtkProcess.bcCellsListFunction(modDrn,'drnCells',listHexaSequence,modDis,modBas)[0]

### Definition of WEL cells values '1' as List
#listWelCellsIO = vtkProcess.bcCellsListFunction(modWel,'welCells',listHexaSequence,modDis,modBas)[0]

### Definition of RIV cells values '1' as List
#listRivCellsIO = vtkProcess.bcCellsListFunction(modRiv,'rivCells',listHexaSequence,modDis,modBas)[0]

### Water Tables on Cell
listWaterTableCell = vtkProcess.listWaterTableCellFunction(modDis,wtObject)

<br/>
<br/>
<br/>
___

# VTK Creation

## Hydraulic conductivity distribution

In [21]:
vtuFileName = '../vtuFiles/'+modName

In [22]:
vtkText = open(vtuFileName+'_DIS_HK.vtu','w')

vtkFunctions.printHeader(vtkText,len(vertexXYZPoints),len(listHexaSequence))

vtkFunctions.printCellData(vtkText,'HK',listCellHK)

vtkFunctions.printPointDefinition(vtkText,vertexXYZPoints)

vtkFunctions.printCellHexaConnectivityOffsetType(vtkText,listHexaSequence)

vtkFunctions.printFooter(vtkText)

vtkText.close()

In [23]:
vtkText = open(vtuFileName+'_DIS_VKA.vtu','w')

vtkFunctions.printHeader(vtkText,len(vertexXYZPoints),len(listHexaSequence))

vtkFunctions.printCellData(vtkText,'VKA',listCellVKA)

vtkFunctions.printPointDefinition(vtkText,vertexXYZPoints)

vtkFunctions.printCellHexaConnectivityOffsetType(vtkText,listHexaSequence)

vtkFunctions.printFooter(vtkText)

vtkText.close()

## Heads on Vertex and Cells VTK

In [24]:
vtkText = open(vtuFileName+'_HD_Heads.vtu','w')

vtkFunctions.printHeader(vtkText,len(vertexXYZPoints),len(listHexaSequence))

vtkFunctions.printPointData(vtkText,'VertexHeads',listVertexHead)

vtkFunctions.printCellData(vtkText,'CellHeads',listCellHead)

vtkFunctions.printPointDefinition(vtkText,vertexXYZPoints)

vtkFunctions.printCellHexaConnectivityOffsetType(vtkText,listHexaSequence)

vtkFunctions.printFooter(vtkText)

vtkText.close()

In [25]:
#vtkText = open(vtuFileName+'_HD_Ucn.vtu','w')

#vtkFunctions.printHeader(vtkText,len(vertexXYZPoints),len(listHexaSequence))

#vtkFunctions.printPointData(vtkText,'VertexUcn',listVertexUcn)

#vtkFunctions.printCellData(vtkText,'CellUcn',listCellUcn)

#vtkFunctions.printPointDefinition(vtkText,vertexXYZPoints)

#vtkFunctions.printCellHexaConnectivityOffsetType(vtkText,listHexaSequence)

#vtkFunctions.printFooter(vtkText)

#vtkText.close()

## Water Table VTK

In [26]:
vtkText = open(vtuFileName+'_HD_WaterTable.vtu','w')

vtkFunctions.printHeader(vtkText,len(vertexWaterTableXYZPoints),len(listWaterTableCell))

vtkFunctions.printCellData(vtkText,'WaterTableElev',listWaterTableCell)

vtkFunctions.printPointDefinition(vtkText,vertexWaterTableXYZPoints)

vtkFunctions.printCellQuadConnectivityOffsetType(vtkText,listLayerQuadSequence)

vtkFunctions.printFooter(vtkText)

vtkText.close()

## DRN Package VTK

In [27]:
vtkText = open(vtuFileName+'_BC_DRNCells.vtu','w')

vtkFunctions.printHeader(vtkText,len(vertexXYZPoints),len(listDrnCellsHexaSecuence))

vtkFunctions.printCellData(vtkText,'DrnCells',listDrnCellsIO)

vtkFunctions.printPointDefinition(vtkText,vertexXYZPoints)

vtkFunctions.printCellHexaConnectivityOffsetType(vtkText,listDrnCellsHexaSecuence)

vtkFunctions.printFooter(vtkText)

vtkText.close()

## WEL Package VTK

In [28]:
#vtkText = open(vtuFileName+'_BC_WELCells.vtu','w')

#vtkFunctions.printHeader(vtkText,len(vertexXYZPoints),len(listWelCellsHexaSecuence))

#vtkFunctions.printCellData(vtkText,'WELCells',listWelCellsIO)

#vtkFunctions.printPointDefinition(vtkText,vertexXYZPoints)

#vtkFunctions.printCellHexaConnectivityOffsetType(vtkText,listWelCellsHexaSecuence)

#vtkFunctions.printFooter(vtkText)

#vtkText.close()

## RIV Package VTK

In [29]:
#vtkText = open(vtuFileName+'_BC_RIVCells.vtu','w')

#vtkFunctions.printHeader(vtkText,len(vertexXYZPoints),len(listRivCellsHexaSecuence))

#vtkFunctions.printCellData(vtkText,'RIVCells',listRivCellsIO)

#vtkFunctions.printPointDefinition(vtkText,vertexXYZPoints)

#vtkFunctions.printCellHexaConnectivityOffsetType(vtkText,listRivCellsHexaSecuence)

#vtkFunctions.printFooter(vtkText)

#vtkText.close()